In [1]:
import requests
from bs4 import BeautifulSoup
import csv
import time
import random
import re
from IPython.display import clear_output
import concurrent.futures
from fake_useragent import UserAgent
import cloudscraper
import subprocess

In [2]:
soloPageUrl = 'https://www.yellow-pages.ph'

In [3]:
def add_spaces_after_punctuation(text):
    return re.sub(r'([,.;:!?)])([^\s])', r'\1 \2', text)

In [4]:
def scrape_business(business):
    
    reviews = []
    
    # Access Page
    about = business.div.h2.a['href']

    # Page source
    pageUrl = soloPageUrl + about
    headers = {'User-Agent': UserAgent().random}
    scraper = cloudscraper.create_scraper() # create a Cloudscraper instance
    aboutUrl = scraper.get(pageUrl, headers=headers).text # use the scraper to get the page source
    soupPage = BeautifulSoup(aboutUrl, 'lxml')
    singlePage = soupPage.find_all('div', class_='yp-container')
    
    for page in singlePage:
        time.sleep(random.uniform(1, 3)) # add random delay between requests
        review_text = page.find('div', class_='yp-see-morex text-break')
        if review_text:
            reviews.append(review_text.text.replace('\n', ''))
        else:
            reviews.append('')

    time.sleep(random.uniform(1, 3))
    # Search for name
    name = business.find('h2', class_='search-tradename').text.replace('\n', '')

    time.sleep(random.uniform(1, 3))
    # Search for address
    address = business.find('span', class_='ellipsis').text

    time.sleep(random.uniform(1, 3))
    # Search for mobile number
    mobile = business.find('a', class_='btn btn-yp-default mr-2 biz-btn-call yp-click')
    mobileNum = mobile['data-phone'] if mobile else ''

    time.sleep(random.uniform(1, 3))
    # Search for average rating
    star_average = page.find('div', class_='rating-num')
    if star_average:
        star_average = star_average.text.replace('\n', '')
    else:
        star_average = ''

    if reviews:
        writer.writerow([name, address, mobileNum, star_average, reviews.pop(0)])
    else:
        writer.writerow([name, address, mobileNum, star_average, ''])

steps = 90
start_page = input("Enter start value: ")
start_page = int(start_page)
headers = {'User-Agent': UserAgent().random}

with open("broker.csv", "a", newline="") as file:
    writer = csv.writer(file)

    for i in range(start_page, steps+1):
        htmlText = f'https://www.yellow-pages.ph/category/residential-properties/page-{i}'
        scraper = cloudscraper.create_scraper()
        try:
            getHtml = scraper.get(htmlText, headers=headers).text
            soup = BeautifulSoup(getHtml, 'lxml')
            scraped = soup.find_all('div', class_='search-listing')
            if not scraped:
                raise Exception("Refresh IP, change start value, and try again.")
        except Exception as e:
            clear_output()
            print(f"Blocked IP and the final step was {i}. {e}")
            break

        print(" ========================================================================")
        print(" URL: ", htmlText)
        print(f" Step {i} scraping...")

        # get list of businesses for this page
        businesses = soup.find_all('div', class_='search-listing')

        with concurrent.futures.ThreadPoolExecutor() as executor:
            results = [executor.submit(scrape_business, business) for business in businesses]
            time.sleep(random.uniform(1, 3)) # add random delay between requests

        print(f" Step {i} finished successfully.")
        print(" Displaying results...")
        print(" ========================================================================")
        print(" Page", i, "& step", steps)
        print(" Names of businesses: ", "\n")
        for business in businesses:
            name = business.find('h2', class_='search-tradename').text.replace('\n', '')
            print(" ", name)
        print(" ========================================================================")
        time.sleep(8)
        clear_output()


Blocked IP and the final step was 31. Refresh IP, change start value, and try again.
